In [ ]:
import os
os.chdir('../')

In [ ]:
from src.utils.helper import read_yaml, create_directories
from src.constants import *
from src import logger

from urllib.parse import urlencode
from dataclasses import dataclass
import urllib.request as request
from pathlib import Path


In [13]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    base_source_url: str
    data_path: Path
    url_params: list
    target_dir: Path

In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILEPATH,
                 params_filepath = PARAMS_FILEPATH,
                 schema_filepath = SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            base_source_url = config.base_source_url,
            data_path= config.data_path,
            url_params = config.url_params,
            target_dir = config.target_dir
        )

        return data_ingestion_config

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.data_path):
            params = urlencode(self.config.url_params, doseq=True)
            url = f"{self.config.base_source_url}?{params}"
            print(url)
            filename, headers = request.urlretrieve(
                url = url,
                filename = self.config.data_path
            )
            logger.info(f'{filename} download! with following info: \n{headers}')
        else:
            logger.info("File already exists")

    def get_daily_data(self):
        pass

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    raise e

[2025-05-14 09:48:00,038: INFO: helper: yaml file: config\config.yaml loaded successfully]
[2025-05-14 09:48:00,045: INFO: helper: yaml file: params.yaml loaded successfully]
[2025-05-14 09:48:00,046: INFO: helper: yaml file: schema.yaml loaded successfully]
[2025-05-14 09:48:00,052: INFO: helper: created directory at: artifacts]
[2025-05-14 09:48:00,058: INFO: helper: created directory at: artifacts/data_ingestion]
https://api.open-meteo.com/v1/forecast?latitude=-7.9797&longitude=112.6304&daily=temperature_2m_max&daily=temperature_2m_min&daily=weather_code&daily=apparent_temperature_max&daily=apparent_temperature_min&daily=wind_speed_10m_max&daily=wind_gusts_10m_max&daily=wind_direction_10m_dominant&daily=shortwave_radiation_sum&daily=uv_index_max&daily=daylight_duration&daily=sunshine_duration&daily=et0_fao_evapotranspiration&start_date=2025-02-13&end_date=2025-05-20&format=csv
[2025-05-14 09:48:03,349: INFO: 195169852: artifacts/data_ingestion/weather_data.csv download! with followi